In [44]:
# %%
import os

# Get the current working directory
current_directory = os.getcwd()

# Construct the full path to the file
file_path = os.path.join(current_directory, "Learning Set", "result.txt")

# Open the file and read its contents
with open(file_path, "r") as file:
    text = file.read()


tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print (text[:52])
print (tokens[:52])




ܘܡܠܟܐ ܕܘܝܕ ܣܐܒ ܘܥܠ ܒܫܢܝܐ ܘܡܟܣܝܢ ܗܘܘ ܠܗ ܒܠܒܘܫܐ ܘܠܐ ܫܚ
[220, 152, 220, 161, 220, 160, 220, 159, 220, 144, 32, 220, 149, 220, 152, 220, 157, 220, 149, 32, 220, 163, 220, 144, 220, 146, 32, 220, 152, 220, 165, 220, 160, 32, 220, 146, 220, 171, 220, 162, 220, 157, 220, 144, 32, 220, 152, 220, 161, 220, 159, 220]


In [45]:
import regex as re
pattern = re.compile(r""" ?ܘ(?=\p{L}+)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

text_words = re.findall(pattern, text) # devide the text into words according to gpt2pat pattern
print(text_words[:52])
print(type(text_words))
words_tokens = []
for i in text_words:
    token_word = i.encode("utf-8")
    token = list(map(int, token_word))
    words_tokens.append(token)
print(words_tokens[:2])



['ܘ', 'ܡܠܟܐ', ' ܕܘܝܕ', ' ܣܐܒ', ' ܘ', 'ܥܠ', ' ܒܫܢܝܐ', ' ܘ', 'ܡܟܣܝܢ', ' ܗܘܘ', ' ܠܗ', ' ܒܠܒܘܫܐ', ' ܘ', 'ܠܐ', ' ܫܚܢ', ' ܘ', 'ܐܡܪܘ', ' ܠܗ', ' ܥܒܕܘܗܝ', ' ܗܐ', ' ܥܒܕܝܟ', ' ܩܕܡܝܟ', ' ܢܒܥܘܢ', ' ܠܡܪܢ', ' ܡܠܟܐ', ' ܥܠܝܡܬܐ', ' ܒܬܘܠܬܐ', ' ܘ', 'ܬܩܘܡ', ' ܩܕܡ', ' ܡܠܟܐ', ' ܘ', 'ܬܗܘܐ', ' ܠܗ', ' ܡܫܡܫܢܝܬܐ', ' ܘ', 'ܬܫܟܒ', ' ܒܥܘܒܟ', ' ܘ', 'ܢܫܚܢ', ' ܠܡܪܢ', ' ܡܠܟܐ', ' ܘ', 'ܒܥܘ', ' ܥܠܝܡܬܐ', ' ܕܫܦܝܪܐ', ' ܒܟܠܗ', ' ܬܚܘܡܐ', ' ܕܐܝܣܪܝܠ', ' ܘ', 'ܐܫܟܚܘ', ' ܠܐܒܝܫܓ']
<class 'list'>
[[220, 152], [220, 161, 220, 160, 220, 159, 220, 144]]


In [46]:
def get_stats(ids):
    counts = {}
    for id in ids:    
        for pair in zip(id, id[1:]): # Pythonic way to iterate consecutive elements
            if pair[1] != 220:  # Check to avoid pairs ending with 220
                counts[pair] = counts.get(pair, 0) + 1
    return counts


def simple_get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        if pair[1] != ord(' ') and pair[1] != 220:  # Check to avoid pairs ending with 220
            counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(words_tokens)
print(stats)
# print(sorted(((v,k) for k,v in stats.items()), reverse=True))

{(220, 152): 33997, (220, 161): 23343, (220, 160): 25917, (220, 159): 11954, (220, 144): 44750, (220, 149): 20078, (220, 157): 29128, (220, 163): 4775, (220, 146): 17886, (220, 165): 10119, (220, 171): 7946, (220, 162): 26463, (220, 151): 16017, (220, 154): 7124, (220, 170): 18226, (220, 169): 5955, (220, 172): 17577, (220, 166): 5097, (220, 147): 3309, (220, 153): 2240, (220, 155): 3273, (220, 168): 1378}


In [47]:
top_pair = max(stats, key=stats.get)

In [48]:
def merge(ids, pair, idx):
  # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
  newids = []
  for sublist in ids:
        i = 0
        new_sublist = []
        while i < len(sublist):
            # if we are not at the very last position AND the pair matches, replace it
            if i < len(sublist) - 1 and sublist[i] == pair[0] and sublist[i + 1] == pair[1]:
                new_sublist.append(idx)
                i += 2
            else:
                new_sublist.append(sublist[i])
                i += 1
        newids.append(new_sublist)
  return newids

def simple_merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

def length(ids):
    return sum(len(id) for id in ids)

print("length:", length(words_tokens))
tokens2 = merge(words_tokens, top_pair, 256)
print(tokens2[:52])
print("length:", length(tokens2))

length: 751752
[[220, 152], [220, 161, 220, 160, 220, 159, 256], [32, 220, 149, 220, 152, 220, 157, 220, 149], [32, 220, 163, 256, 220, 146], [32, 220, 152], [220, 165, 220, 160], [32, 220, 146, 220, 171, 220, 162, 220, 157, 256], [32, 220, 152], [220, 161, 220, 159, 220, 163, 220, 157, 220, 162], [32, 220, 151, 220, 152, 220, 152], [32, 220, 160, 220, 151], [32, 220, 146, 220, 160, 220, 146, 220, 152, 220, 171, 256], [32, 220, 152], [220, 160, 256], [32, 220, 171, 220, 154, 220, 162], [32, 220, 152], [256, 220, 161, 220, 170, 220, 152], [32, 220, 160, 220, 151], [32, 220, 165, 220, 146, 220, 149, 220, 152, 220, 151, 220, 157], [32, 220, 151, 256], [32, 220, 165, 220, 146, 220, 149, 220, 157, 220, 159], [32, 220, 169, 220, 149, 220, 161, 220, 157, 220, 159], [32, 220, 162, 220, 146, 220, 165, 220, 152, 220, 162], [32, 220, 160, 220, 161, 220, 170, 220, 162], [32, 220, 161, 220, 160, 220, 159, 256], [32, 220, 165, 220, 160, 220, 157, 220, 161, 220, 172, 256], [32, 220, 146, 220, 172, 22

In [49]:


# ---
vocab_size = 6000 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(words_tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

merging (220, 144) into a new token 256


merging (220, 152) into a new token 257
merging (220, 157) into a new token 258
merging (220, 162) into a new token 259
merging (220, 160) into a new token 260
merging (220, 161) into a new token 261
merging (220, 149) into a new token 262
merging (220, 170) into a new token 263
merging (220, 146) into a new token 264
merging (220, 172) into a new token 265
merging (220, 151) into a new token 266
merging (32, 257) into a new token 267
merging (220, 159) into a new token 268
merging (32, 262) into a new token 269
merging (220, 165) into a new token 270
merging (32, 256) into a new token 271
merging (32, 260) into a new token 272
merging (220, 171) into a new token 273
merging (32, 261) into a new token 274
merging (220, 154) into a new token 275
merging (32, 264) into a new token 276
merging (220, 169) into a new token 277
merging (257, 259) into a new token 278
merging (220, 166) into a new token 279
merging (258, 259) into a new token 280
merging (220, 163) into a new token 281
mergin

In [50]:
print("tokens length:", length(words_tokens))
print("ids length:", length(ids))
print(f"compression ratio: {length(words_tokens) / length(ids):.2f}X")

tokens length: 751752
ids length: 111069
compression ratio: 6.77X


### Generate Vocab


In [51]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

vocab_converted= {idx: vocab[idx].decode("utf-8") for idx in vocab if idx > 255}

merges_converted = {str(key): value for key, value in merges.items()}

In [55]:
print(vocab_converted)
print(vocab)
print(merges_converted)

{256: 'ܐ', 257: 'ܘ', 258: 'ܝ', 259: 'ܢ', 260: 'ܠ', 261: 'ܡ', 262: 'ܕ', 263: 'ܪ', 264: 'ܒ', 265: 'ܬ', 266: 'ܗ', 267: ' ܘ', 268: 'ܟ', 269: ' ܕ', 270: 'ܥ', 271: ' ܐ', 272: ' ܠ', 273: 'ܫ', 274: ' ܡ', 275: 'ܚ', 276: ' ܒ', 277: 'ܩ', 278: 'ܘܢ', 279: 'ܦ', 280: 'ܝܢ', 281: 'ܣ', 282: 'ܬܐ', 283: 'ܪܝ', 284: ' ܥ', 285: 'ܓ', 286: 'ܛ', 287: 'ܢܐ', 288: ' ܗ', 289: 'ܠܐ', 290: 'ܪܐ', 291: 'ܡܐ', 292: ' ܡܢ', 293: 'ܙ', 294: ' ܢ', 295: 'ܝܬ', 296: 'ܠܗ', 297: ' ܬ', 298: 'ܝܐ', 299: 'ܗܝ', 300: ' ܕܐ', 301: ' ܟ', 302: ' ܚ', 303: 'ܪܝܐ', 304: ' ܠܐ', 305: 'ܡܪ', 306: 'ܝܟ', 307: ' ܫ', 308: ' ܐܢ', 309: ' ܩ', 310: ' ܕܡ', 311: 'ܗܘܢ', 312: 'ܘܗܝ', 313: ' ܥܠ', 314: 'ܒܕ', 315: ' ܗܘ', 316: 'ܨ', 317: 'ܠܟ', 318: 'ܝܘ', 319: 'ܫܐ', 320: ' ܓ', 321: 'ܕܡ', 322: ' ܠܡ', 323: 'ܒܐ', 324: 'ܕܐ', 325: 'ܪܥ', 326: 'ܘܬ', 327: 'ܒܪ', 328: ' ܡܪܝܐ', 329: 'ܗܘ', 330: 'ܛܠ', 331: ' ܣ', 332: 'ܝܡ', 333: ' ܠܗ', 334: ' ܦ', 335: 'ܘܪ', 336: 'ܢܝ', 337: 'ܘܬܐ', 338: 'ܝܣ', 339: 'ܠܟܐ', 340: ' ܕܥ', 341: ' ܐܝܟ', 342: ' ܪ', 343: 'ܐܡܪ', 344: 'ܝܠ', 345: ' ܕܒ', 346: ' ܠܟ

In [54]:
import json


file_path_voc = os.path.join(current_directory, f"vocabulary_{vocab_size}.json")
file_path_merge = os.path.join(current_directory, f"merges_{vocab_size}.json")


with open(file_path_voc, "w") as file:
    json.dump(vocab_converted, file, indent=4)
with open(file_path_merge, "w") as file:
    json.dump(merges_converted, file, indent=4)   

    
print(f"Vocabulary and merges saved to {file_path}")

Vocabulary and merges saved to /home/zhanchen/Dropbox/Projects/Tokenizer/Learning Set/result.txt
